Mohammed Khan

012804870



In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas
import re

def findTables(url):
  res = requests.get(url)
  comm = re.compile("<!--|-->")
  soup = bs4.BeautifulSoup(comm.sub("", res.text), 'lxml')
  divs = soup.findAll('div', id = "content")
  divs = divs[0].findAll("div", id=re.compile("^all"))
  ids = []
  for div in divs:
     searchme = str(div.findAll("table"))
     x = searchme[searchme.find("id=") + 3: searchme.find(">")]
     x = x.replace("\"", "")
     if len(x) > 0:
        ids.append(x)
  return(ids)
  
def pullTable(url, tableID, header = True):
  res = requests.get(url)
  ## Work around comments
  comm = re.compile("<!--|-->")
  soup = bs4.BeautifulSoup(comm.sub("", res.text), 'lxml')
  tables = soup.findAll('table', id = tableID)
  data_rows = tables[0].findAll('tr')
  game_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
     for i in range(len(data_rows))
     ]
  data = pandas.DataFrame(game_data)
  if header == True:
     data_header = tables[0].findAll('thead')
     data_header = data_header[0].findAll("tr")
     data_header = data_header[0].findAll("th")
     header = []
     for i in range(len(data.columns)):
        header.append(data_header[i].getText())
     data.columns = header
     data = data.loc[data[header[0]] != header[0]]
  data = data.reset_index(drop = True)
  return(data)
  
def pullTable1(url, tableID, header = True):
  res = requests.get(url)
  ## Work around comments
  comm = re.compile("<!--|-->")
  soup = bs4.BeautifulSoup(comm.sub("", res.text), 'lxml')
  tables = soup.findAll('table', id = tableID)
  data_rows = tables[0].findAll('tr')
  game_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
     for i in range(len(data_rows))
     ]
    
  data = pandas.DataFrame(game_data)
  if header == True:
     data_header = tables[0].findAll('thead')
     data_header = data_header[0].findAll("tr")
     data_header = data_header[1].findAll("th")
     header = []
     for i in range(len(data.columns)):
        header.append(data_header[i].getText())
     data.columns = header
     data = data.loc[data[header[0]] != header[0]]
  data = data.reset_index(drop = True)
  return(data)
  
url = 'https://www.pro-football-reference.com/teams/kan/2018.htm'
page = requests.get('https://www.pro-football-reference.com/teams/kan/2018.htm')
soup = BeautifulSoup(page.text, 'html.parser')
print(findTables(url))
passersDat = pullTable(url, "passing")
passersDat = pandas.DataFrame(passersDat.iloc[:,[0,1, 2,3,4,5,7,8,9,10,11,12,13,14,16,18,19,21,22]])
test_list = ["No.","Player", "Age","Pos","Games","Started","Cmp","Att","Cmp%","Yds","Td","Td%","Int","Int%","Y/A","Y/C","Y/G",
              "QBR","Sacked"]
passersDat.columns = test_list
for p in test_list:
  print(p,': ',passersDat.at[0,p])
print()
print(passersDat.at[0,"QBR"])

rbs_wrsDat = pullTable1(url,"rushing_and_receiving")
rbs_wrsDat = pandas.DataFrame(rbs_wrsDat.iloc[:,[0,1, 2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,19,20,21,26]])
test2_list = ["No.","Player", "Age","Pos","Games","Started",
        "RushAtt","RushYds","RushTd","RushYds/Att","RushYds/Game","Att/Game",
        "RecTargets","RecCatch","RecYds","Yds/Rec","RecTd","Rec/Game","Yds/Game","Rec%",
              "Fmb"]
#0 No.  Player Age Pos G   GS  5
#6 Att  Yds TD  Lng Y/A Y/G A/G 12
#13 Tgt Rec Yds Y/R TD  Lng R/G Y/G Ctch%   21
#22 Touch   Y/Tch   YScm    RRTD    Fmb 26

rbs_wrsDat.columns = test2_list

for p in test2_list:
  print(p,': ',rbs_wrsDat.at[1,p])
print()

url = 'https://www.pro-football-reference.com/years/2018/week_1.htm'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
#print(soup)
next_soup = soup.find('div', {'class' :'game_summaries'})

second_serving = next_soup.find_all('table'),{'class':'teams'}
#print(second_serving[0].find_all('td', {'class' :'right gamelink'}))
#for t in second_serving[:
#    if '/boxscores/' in str(t):
#        continue
t_list = []
#print(len(second_serving[0]))
for t in second_serving[0]:
   for p in t:
      for d in p:
        for c in d:
           if '/boxscores/' in str(c):
              result = re.search('boxscores/(.*).htm',str(c))
              t_list.append(result.group(1))
              continue
print(t_list)


['kan_current_injuries', 'team_stats', 'games', 'team_conversions', 'passing', 'rushing_and_receiving', 'returns', 'kicking', 'defense', 'scoring', 'team_td_log', 'opp_td_log']
No. :  15
Player :  Patrick Mahomes
Age :  23
Pos :  QB
Games :  7
Started :  7
Cmp :  163
Att :  251
Cmp% :  64.9
Yds :  2223
Td :  22
Td% :  8.8
Int :  5
Int% :  2.0
Y/A :  8.9
Y/C :  13.6
Y/G :  317.6
QBR :  85.9
Sacked :  8

85.9
No. :  27
Player :  Kareem Hunt
Age :  23
Pos :  RB
Games :  7
Started :  7
RushAtt :  118
RushYds :  542
RushTd :  5
RushYds/Att :  4.6
RushYds/Game :  77.4
Att/Game :  16.9
RecTargets :  21
RecCatch :  15
RecYds :  226
Yds/Rec :  15.1
RecTd :  4
Rec/Game :  2.1
Yds/Game :  32.3
Rec% :  71.4%
Fmb :  0

['201809060phi', '201809090clt', '201809090cle', '201809090rav', '201809090nyg', '201809090nwe', '201809090nor', '201809090min', '201809090mia', '201809090sdg', '201809090car', '201809090den', '201809090crd', '201809090gnb', '201809100det', '201809100rai']
